# Ingest Summoner Ranks API to Bronze
This notebook loads the summoner ranks data from the League of Legends API. Cf. detail on the API by Riot Games on: https://developer.riotgames.com/

It requires the list of summoners for which we want to get data as input.

The list of summoners from which you want to get data was loaded as a csv file was created for that and manually ingested in the Files section of the Bronze Lakehouse. The csv file contains summoner name (which can change) ,the Player Universally Unique ID (puuid, which cannot change) and the Summoner ID (which is needed for some API calls). The puuid was fetched using the summoner API : https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/"summoner-name"


### 1. General configuration

In [ ]:
# Import libraries
from trident_token_library_wrapper import PyTridentTokenLibrary as tl
import requests as r
import json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType
from pyspark.sql import Row
from datetime import datetime

In [ ]:
# Load secrets from Azure Key Vault
key_vault_name = 'testasa-akv-dev-001'
key_name = 'lol-api-key'
access_token = mssparkutils.credentials.getToken("keyvault")
api_key = tl.get_secret_with_token(f"https://{key_vault_name}.vault.azure.net/", key_name, access_token)

# Define API endpoint and parameters
base_url = 'https://euw1.api.riotgames.com/lol/'

### 2. Load Summoner Ranks

In [ ]:
# Read Summoners.csv into a DataFrame
print("Reading summoners csv file into a DataFrame")
df_summoners = spark.read.option("header", "true").option("inferSchema", "true").option("delimiter", ";").csv("Files/raw_data/Summoners.csv")
print("Finished reading summoners csv file into a DataFrame, found", df_summoners.count(), "summoners")

# Make a list out of the Summoner IDs to iterate on Summoner IDs
summonerId_list = df_summoners.select('SummonerId').collect()

In [ ]:
# Make API calls to get ranks of each summoner and collect responses

print("Starting API calls to get summoner ranks...")

all_ranks = []

for summoner_row in summonerId_list:
    print(f"Getting data for summonerId: {summoner_row[0]} ")
    summoner_id = summoner_row[0]
    params = {'api_key': api_key}
    summoner_rank = r.get(f"{base_url}league/v4/entries/by-summoner/{summoner_id}", params=params)

    if summoner_rank.status_code == 200:
        response = summoner_rank.json()

        for response_dict in response:
            if response_dict["queueType"] != "CHERRY": # exclude placement games
                all_ranks.append(Row(**response_dict))

        print(f"Successfully fetched data for summonerId: {summoner_id}")

    else:
        print(f"Failed to fetch data for summonerId: {summoner_id}")

In [ ]:
# Define the schema for the Summoner ranks
schema = StructType([
    StructField("leagueId", StringType(), True),
    StructField("queueType", StringType(), True),
    StructField("tier", StringType(), True),
    StructField("rank", StringType(), True),
    StructField("summonerId", StringType(), True),
    StructField("leaguePoints", IntegerType(), True),
    StructField("wins", IntegerType(), True),
    StructField("losses", IntegerType(), True),
    StructField("veteran", BooleanType(), True),
    StructField("inactive", BooleanType(), True),
    StructField("freshBlood", BooleanType(), True),
    StructField("hotStreak", BooleanType(), True)
])

# Create DataFrame from the collected rows and schema
df_summoner_ranks = spark.createDataFrame(all_ranks, schema=schema)

# Get current date
current_date = datetime.now()
date_str = current_date.strftime("%Y/%m/%d")

# Save file to appropriate bronze folder; replace old file if it already existed
print(f"Writing summoner ranks data to Bronze Lakehouse in files {date_str}")
df_summoner_ranks.coalesce(1).write.mode("overwrite").json(f'Files/raw_data/summoner_ranks/{date_str}')